<a href="https://colab.research.google.com/github/fernanda993/hello-word/blob/master/coleta_dados_e_urls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from time import sleep

In [ ]:
# primeiras 6 paginas do site
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', '_Desde_241']
zonas = ['norte', 'sul', 'leste', 'oeste']

In [ ]:
def coleta_dados(url, zona):
  conteudo = requests.get(url).content
  sopa = BeautifulSoup(conteudo)

  lista_precos = []
  lista_areas_quartos = []
  lista_quartos = []
  lista_dimensoes = []
  lista_zonas = []

  re_quartos = '(.) quarto|quartos'
  re_dimensoes = '(.*) m²'

  precos = sopa.find_all('span', class_='price-tag-fraction')
  dimensoes_e_quartos = sopa.find_all('ul', class_='ui-search-card-attributes ui-search-item__group__element')

  for elemento in precos:
    preco = re.findall('<span class="price-tag-fraction">(.*)</span>', str(elemento))
    lista_precos.append(float(preco[0])*1000)   

  for elemento in dimensoes_e_quartos:
    lista_areas_quartos.append(elemento.text)

  for elemento in lista_areas_quartos:
    quarto = re.findall(re_quartos, str(elemento))
    dimensao = re.findall(re_dimensoes, str(elemento).replace(',', ''))

    if quarto == []:
      quarto = np.nan
    else:
      quarto = quarto[0]
    
    if dimensao == []:
      dimensao = np.nan
    else:
      dimensao = dimensao[0]
  
    lista_quartos.append(quarto)
    lista_dimensoes.append(dimensao)
    lista_zonas.append(zona)

  dados = {'zona': lista_zonas,
           'precos':lista_precos,
           'quartos':lista_quartos,
           'areas': lista_dimensoes}
  
  df = pd.DataFrame(dados)
  return df

In [ ]:
lista_dfs = []

for zona in zonas:
  for pagina in paginas:
    url = f'https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-{zona}/{pagina}'
    print('Coletando dados de', url)
    df = coleta_dados(url, zona)
    lista_dfs.append(df)
    sleep(2)
print('Dados coletados!')

Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_49
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_97
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_145
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_193
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_241
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zo

In [ ]:
df.head()

,zona,precos,quartos,areas
0,oeste,1500.0,3,75
1,oeste,1500.0,3,75
2,oeste,850000.0,1,15
3,oeste,5900.0,NaN,294
4,oeste,2700.0,3,63


In [ ]:
df_ml = pd.concat(lista_dfs)

In [ ]:
df_ml.head()

,zona,precos,quartos,areas
0,norte,2990.0,3,475
1,norte,980000.0,2,55
2,norte,1700.0,2,58
3,norte,1350.0,NaN,28
4,norte,1000.0,3,55


In [ ]:
df_ml.shape

(1152, 4)

In [ ]:
df_ml.tail()

,zona,precos,quartos,areas
43,oeste,2900.0,2,85
44,oeste,1400.0,2,44
45,oeste,1750.0,3,60
46,oeste,2200.0,1,35
47,oeste,1500.0,1,28


In [ ]:
df_ml.to_csv('dados_mercado_livre.csv', index=False)

In [ ]:
pd.read_csv('dados_mercado_livre.csv')

,zona,precos,quartos,areas
0,norte,2990.0,3.0,475.0
1,norte,980000.0,2.0,55.0
2,norte,1700.0,2.0,58.0
3,norte,1350.0,NaN,28.0
4,norte,1000.0,3.0,55.0
...,...,...,...,...
1147,oeste,2900.0,2.0,85.0
1148,oeste,1400.0,2.0,44.0
1149,oeste,1750.0,3.0,60.0
1150,oeste,2200.0,1.0,35.0


In [ ]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1152 entries, 0 to 47
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   zona     1152 non-null   object 
 1   precos   1152 non-null   float64
 2   quartos  1103 non-null   object 
 3   areas    1149 non-null   object 
dtypes: float64(1), object(3)
memory usage: 45.0+ KB


In [ ]:
df_ml.isnull().sum()

zona        0
precos      0
quartos    49
areas       3
dtype: int64

In [ ]:
valor_ponto = '1,055'.replace(',', '.')

In [ ]:
valor_nada = '1,055'.replace(',', '')

In [ ]:
float(valor_ponto) + 2

3.0549999999999997

In [ ]:
float(valor_nada) + 2

1057.0